In [1]:
import torch
import torchvision
import os
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
# transform = transforms.Compose(
#     [transforms.ToTensor(),
#      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

# batch_size = 4

# trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
#                                         download=True, transform=transform)
# trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
#                                           shuffle=True, num_workers=2)

# testset = torchvision.datasets.CIFAR10(root='./data', train=False,
#                                        download=True, transform=transform)
# testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
#                                          shuffle=False, num_workers=2)

# classes = ('plane', 'car', 'bird', 'cat',
#            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

In [3]:
directory = 'processedDataset/'

class StockDataset(Dataset):
    def __init__(self):
        self.data = []
        i = 0
        tempData = os.listdir(directory)
        while i < 16000:
            f = os.path.join(directory, tempData[i])
            if torch.any(torch.isnan(torch.load(f))):
                print(f + " contains nan")
                continue
            else:
                x = torch.load(f).float().to(device)
                x = torch.nn.functional.normalize(x, p=2.0, dim = 1)
                label = int(torch.load(f)[0].sum() > torch.load(f)[1].sum())
                self.data.append([x, label])
                i += 1
                # if filename.split('-')[2].replace(".pt","") == '1':
                #     Y.append(torch.tensor([0,1]).float())
                # else:
                #     Y.append(torch.tensor([1,0]).float())
            

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        return self.data[i][0][0:2], self.data[i][1]

In [4]:
class TestStockDataset(Dataset):
    def __init__(self):
        self.data = []
        i = 0
        tempData = os.listdir(directory)
        tempData.reverse()
        while i < 1600:
            f = os.path.join(directory, tempData[i])
            if torch.any(torch.isnan(torch.load(f))):
                print(f + " contains nan")
                continue
            else:
                x = torch.load(f).float().to(device)
                x = torch.nn.functional.normalize(x, p=2.0, dim = 1)
                label = int(torch.load(f)[0][0].sum() > torch.load(f)[1][0].sum())
                self.data.append([x, label])
                i += 1
                # if filename.split('-')[2].replace(".pt","") == '1':
                #     Y.append(torch.tensor([0,1]).float())
                # else:
                #     Y.append(torch.tensor([1,0]).float())
            

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        return self.data[i][0][0:2], self.data[i][1]

In [5]:
batch_size = 64
stockDataset = StockDataset()
testDataset = TestStockDataset()
trainloader = torch.utils.data.DataLoader(stockDataset, batch_size=batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(testDataset, batch_size=batch_size, shuffle=False)

In [6]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(22 * 2, 2)
        # self.fc2 = nn.Linear(120, 84)
        # self.fc3 = nn.Linear(84, 2)
    def forward(self, x):
        # x = self.pool(F.relu(self.conv1(x)))
        # x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = self.fc1(x)
        # x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))
        # x = self.fc3(x)
        return x


net = Net()
if device == 'cuda':
    net = net.cuda()

In [7]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
optimizer = optim.SGD(net.parameters(), lr=0.003, momentum=0.9)

In [8]:
from cProfile import label, run
import string


for epoch in range(10000):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()
        if device == 'cuda':
            labels = labels.cuda()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
    
    if epoch % 100 == 99:    # print every 2000 mini-batches
        print(f'epoch {epoch + 1}: loss: {running_loss /16000*batch_size}')

print('Finished Training')

epoch 100: loss: 0.5757992094755173
epoch 200: loss: 0.5393297301530838
epoch 300: loss: 0.5181512573957443
epoch 400: loss: 0.5036199244260788
epoch 500: loss: 0.4929525636434555
epoch 600: loss: 0.4846720849275589
epoch 700: loss: 0.47799241042137147
epoch 800: loss: 0.4724190791845322
epoch 900: loss: 0.46778318071365355
epoch 1000: loss: 0.4638344588279724
epoch 1100: loss: 0.46031678378582
epoch 1200: loss: 0.45722998297214507
epoch 1300: loss: 0.45453360152244565
epoch 1400: loss: 0.4520802807807922
epoch 1500: loss: 0.4498594862222671
epoch 1600: loss: 0.44795352482795714
epoch 1700: loss: 0.4461701581478119
epoch 1800: loss: 0.444527016043663
epoch 1900: loss: 0.44305200028419495
epoch 2000: loss: 0.4416123623847961
epoch 2100: loss: 0.44035154628753664
epoch 2200: loss: 0.43914720380306244
epoch 2300: loss: 0.43796308076381685
epoch 2400: loss: 0.43695631849765776
epoch 2500: loss: 0.4360013282299042
epoch 2600: loss: 0.4348953629732132
epoch 2700: loss: 0.43420226669311524
ep

In [15]:
PATH = './cifar_net.pth'
torch.save(net.state_dict(), PATH)

In [16]:
net = Net()
if device == 'cuda':
    net = net.cuda()
net.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [17]:
dataiter = iter(testloader)
stocks, labels = dataiter.next()

In [18]:
outputs = net(stocks)

In [19]:
_, predicted = torch.max(outputs, 1)
print(predicted)
print(labels)

tensor([1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
        1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0,
        0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1])
tensor([0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0,
        0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1,
        0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1])


In [20]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in testloader:
        images, labels = data
        # calculate outputs by running images through the network
        outputs = net(images)
        if device == 'cuda':
            labels = labels.cuda()
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct // total} %')

Accuracy of the network on the test images: 47 %
